In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from numpy.linalg import norm

In [ ]:
from sklearn import preprocessing

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import json

In [ ]:
encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preproc_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_preproc_model = hub.KerasLayer(preproc_url)

In [ ]:
text_test = ['fever/cough', 'cough']
text_preprocessed = bert_preproc_model(text_test)
text_preprocessed.keys()

In [ ]:
text_preprocessed['input_word_ids']

In [ ]:
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
bert_results = bert_model(text_preprocessed)

In [ ]:
bert_results.keys()

In [ ]:
bert_results['sequence_output']

In [ ]:
res=bert_results['pooled_output']

In [ ]:
cos=np.dot(res[0],res[1])/(norm(res[0])*norm(res[1]))

In [ ]:
cos

In [ ]:
df=pd.read_csv('Covid Data/deathcases.csv')

In [ ]:
df

In [ ]:
df['Symptoms']=df['Symptoms'].apply(lambda x: x.lower().strip(' '))

In [ ]:
label_encoder=preprocessing.LabelEncoder()

In [ ]:
df['symptomCode']=label_encoder.fit_transform(df['Symptoms'])

In [ ]:
df['symptomCode']

In [ ]:
label_encoder.classes_

In [ ]:
symptom_list=list(label_encoder.classes_)

In [ ]:
symptom_preproc=bert_preproc_model(symptom_list[1000:])

In [ ]:
symptom_preproc['input_word_ids']

In [ ]:
res=bert_model(symptom_preproc)

In [ ]:
res['pooled_output'][0]

In [ ]:
pooled_out={x:np.array(res['pooled_output'][x] for x in range(122)}

In [ ]:
pooled.update(pooled_out)

In [ ]:
len(pooled)

In [ ]:
df['symptomcodeInverse']=label_encoder.inverse_transform(df['symptomCode'])

In [ ]:
df['symptomcodeInverse']

In [ ]:
df['symptomtensor']=df['symptomCode'].map(lambda x: tfdf.iloc[x].tolist())

In [ ]:
df['symptomtensor'][0]

In [ ]:
file=open('tensor.txt', 'a')

In [ ]:
for x in list(pooled.keys())[:10]:
    tf.io.write_file('tensor.txt', str(pooled[x]))

In [ ]:
tfdf=pd.DataFrame(pooled)

In [ ]:
tfdf=tfdf.transpose()

In [ ]:
tfdf

In [ ]:
tfdf.to_csv('tensor.csv', index=False)

In [ ]:
tfdf=pd.read_csv('tensor.csv')

In [ ]:
wcss=[]

In [ ]:
df

In [ ]:
kmeansDf=df.iloc[:,[-1, -4] ]

In [ ]:
split = pd.DataFrame(kmeansDf['symptomtensor'].to_list(), columns = range(768))

In [ ]:
kmeansDf=pd.concat([kmeansDf, split], axis=1)

In [ ]:
kmeansDf=kmeansDf.drop('symptomtensor', axis=1)

In [ ]:
kmeansDf.columns=kmeansDf.columns.astype(str)

In [ ]:
for i in range(1, 11):
    kmeans=KMeans(n_clusters=i, random_state=0, n_init='auto')
    kmeans.fit(kmeansDf)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss)

In [ ]:
kmeans=KMeans(n_clusters=4, random_state=0, n_init='auto')
kmeans.fit(kmeansDf)

In [ ]:
cluster_labels=kmeans.predict(kmeansDf)

In [ ]:
color=['red', 'blue', 'yellow', 'green', 'black', 'pink']

In [ ]:
df1=pd.read_csv('Covid data/deathcases.csv')

In [ ]:
df1['cluster']=cluster_labels

In [ ]:
df1.to_csv('symptoms_cluster_4.csv')